# Hipótesis 4

#### "Las provincias situadas en el sur de España (aquellas al sur de Madrid) tienden a utilizar menos dispositivos de aprovechamiento de energía renovable que las del norte." 

Las diferencias climáticas entre el norte y sur de España pueden ser una variable crucial a la hora de instalar dispositivos de energía renovable pero no la única. El clima influye en la cantidad de energía que se puede producir pero también en la que se consume.

Además puede influir el ámbito socioeconómico, las provincias con mayor poder adquisitivo podrían gastarse más en energía renovable o las más pobres podrían estar motivadas por el ahorro en la factura de la luz. Otras posibles factores de influencia, pueden ser los tamaños familiares o la edad.

Con estos factores, se pretende agrupar las provincias mediante clustering y ver si se pueden diferenciar entre las del norte y sur de España y las características que las definen.

## Preparación del entorno

In [ ]:
import os 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.metrics import silhouette_score

GOLD_DATA_PATH = os.path.join("..", "..", "data/gold/")

## Datos

Los datos que se usarán están en el dataset de la tarjeta de datos 4: `data_card_4_df.csv`

In [ ]:
# Cargar de los datos
df4 = pd.read_csv(GOLD_DATA_PATH + "data_card_4_df.csv", sep=";", encoding = 'latin')
df4.info()

In [ ]:
# Formateado de la tabla
df4 = df4.drop(columns=["Unnamed: 0"])
df4.set_index("Provincias", inplace=True)
df4.head(3)

### Estandarización de los datos

Cada variable tiene una escala diferente, por lo que es necesario estandarizar los datos para que todas tengan la misma importancia en el análisis.
Se ha elegido el MinMaxScaler para que los valores estén entre 0 y 1, intentando modificar lo menos posible la distribución de los datos.


In [ ]:
# Escalado de los datos a un rango de 0 a 1
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df4)
df4_scaled = pd.DataFrame(scaled_data, columns=df4.columns)

df4_scaled.head(3)

### Análisis de componentes principales (PCA)

Para reducir la dimensionalidad de los datos y poder visualizarlos, se aplicará PCA. La representación se hará tanto en 2D como en 3D.

In [ ]:
# Calculo de PCA
from sklearn.decomposition import PCA

estimator = PCA(n_components=3)
X_pca = estimator.fit_transform(scaled_data)

print("Porcentaje de varianza explicado por cada componente:\n", estimator.explained_variance_ratio_)
pd.DataFrame(np.matrix.transpose(estimator.components_), index=df4.columns)

In [ ]:
#Representación 2D
fig, ax = plt.subplots()
ax.scatter(X_pca[:,0], X_pca[:,1], s=50)

# anotación 
for i in range(0, len(X_pca)):
    ax.annotate(df4.iloc[i, :].name, (X_pca[i, 0], X_pca[i, 1]), fontsize=8)

In [ ]:
# Representación 3D
import plotly.express as px

fig = px.scatter_3d(df4, x=X_pca[:,0], y=X_pca[:,1], z=X_pca[:,2], color=df4.index, color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.show()

## Modelado

### Clustering jerárquico

Para el clustering jerárquico se usará el método de Ward, que minimiza la varianza intra-cluster. Y la métrica de distancia será la euclídea.

In [ ]:
# Creación del dendrograma
link_matrix_avg = linkage(scaled_data, method='ward', metric='euclidean')

plt.figure(figsize=(8, 5))
dendrogram(link_matrix_avg, labels=df4.index)
plt.show()

In [ ]:
# probar varias configuraciones de clusters cambiando n
n = 2
clusters = fcluster(link_matrix_avg, t=n, criterion='maxclust')
coef = silhouette_score(df4_scaled, clusters)
print(f"Coeficiente de silhouette para {n} clusters: {coef:.4f}")

In [ ]:
# visualización del resultado
plt.figure(figsize=(8, 5))
scatter = plt.scatter(X_pca[:,0], X_pca[:,1], s=50, c=clusters, cmap='plasma_r')

# nombres de las provincias 
for i in range(0, len(X_pca)):
    plt.annotate(df4.iloc[i, :].name, (X_pca[i, 0], X_pca[i, 1]), fontsize=8)

handles, labels = scatter.legend_elements()
legend = plt.legend(handles, labels, title="Clusters", bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()

In [ ]:
import plotly.express as px

fig = px.scatter_3d(df4, x=X_pca[:,0], y=X_pca[:,1], z=X_pca[:,2], color=clusters, color_continuous_scale='plasma_r')
fig.show()

## Resultados

Mapa de la península con las provincias coloreadas según el cluster al que pertenecen.   
![img](img/Edad_mediana_2C.png)

In [ ]:
df4_scaled['Cluster'] = clusters
df4_melted = df4_scaled.melt(id_vars="Cluster", var_name="Variable", value_name="Valor")
plt.figure(figsize=(12, 6))
sns.boxplot(x="Variable", y="Valor", hue="Cluster", data=df4_melted, palette="Set3")
plt.title("Distribución de las variables por cluster (estandarizadas)")
plt.xticks(rotation=90)
plt.show()

**Extra**

Este sería el resultado si se hubieran elegido 3 clusters. Es difícil interpretarlo.

![img](img/Edad_mediana_3C.png)